In [9]:
import pandas as pd
import yfinance as yf
import plotly.express as px

In [6]:
history = pd.DataFrame()
history['ACA.PA'] = yf.Ticker('ACA.PA').history('1y')['Close'].tz_localize(None)
history['DG.PA'] = yf.Ticker('DG.PA').history('1y')['Close'].tz_localize(None)
history['MC.PA'] = yf.Ticker('MC.PA').history('1y')['Close'].tz_localize(None)

benchmark = pd.DataFrame(yf.Ticker('^GSPC').history('1y')['Close'].tz_localize(None))
riskfree = pd.DataFrame(yf.Ticker('^TNX').history('1y')['Close'].tz_localize(None))

In [11]:
asset = history['ACA.PA']

# Calculate the cumulative percentage change in price or returns
index = 100 * (1 + history.pct_change()).cumprod()

# Identify the peaks and calculate drawdowns
peaks = index.cummax()
drawdowns = (index - peaks) / peaks

In [ ]:
def drawdowns_comparison(self, portfolio_history, benchmark):
    """
    Generate a performance comparison chart with portfolio, benchmark, and individual assets.
    
    Parameters:
    -----------
    portfolio_history : pd.DataFrame
        Historical portfolio data with assets as columns
    benchmark : pd.Series
        Benchmark time series data
    
    Returns:
    --------
    str : HTML string of the chart
    """
    # Prepare data
    asset_names = portfolio_history.columns.tolist()
    comparison_data = portfolio_history.copy()
    comparison_data["Portfolio"] = portfolio_history.sum(axis=1)
    comparison_data['Benchmark'] = benchmark
    comparison_data = comparison_data.dropna()

    # Calculate the cumulative percentage change in price or returns
    indexed_data = 100 * (1 + comparison_data.pct_change()).cumprod()

    # Identify the peaks and calculate drawdowns
    peaks = indexed_data.cummax()
    indexed_data = (indexed_data - peaks) / peaks
    
    # Create figure
    fig = go.Figure()
    
    # Add portfolio line
    fig.add_trace(go.Scatter(
        x=indexed_data.index,
        y=indexed_data["Portfolio"],
        line_color="#1a3a8f",
        mode='lines',
        name='Portfolio'
    ))
    
    # Add benchmark line
    fig.add_trace(go.Scatter(
        x=indexed_data.index,
        y=indexed_data["Benchmark"],
        line_color='#d62728',
        mode='lines',
        name='Benchmark'
    ))
    
    # Add individual assets (initially hidden)
    color_palette = self._generate_color_palette("#1a3a8f", len(asset_names) + 1)
    for asset, color in zip(asset_names, color_palette[1:]):
        fig.add_trace(go.Scatter(
            x=indexed_data.index,
            y=indexed_data[asset],
            line=dict(color=color, dash='dashdot'),
            mode='lines',
            name=asset,
            visible='legendonly'
        ))
    
    return self._apply_standard_layout(fig).to_html(config={'displaylogo': False})